# Temperature dependent elastic constants

## Background

$$C_{ijkl} = \frac{1}{V} \frac{\partial^2 U}{\partial \varepsilon_{ij}\partial \varepsilon_{kl}}$$

$$U(T) = \frac{V}{2}C_{ijkl}(T)\varepsilon_{ij}\varepsilon_{kl}$$

$$\sigma_{ij} = C_{ijkl}{\varepsilon_{kl}}$$

### How to get $U$ or $\sigma$

- MD
- Quasi-Harmonic

## Tasks

- Get $a_0$ from potential
- Lattice parameter (as a function of T)
  - MD
    - NVT
    - NPT
  - QH
- Calculate $U$ or $\sigma$ for various $\varepsilon$
  - MD: Equilibriate and average with LAMMPS
  - QH: Get strains from Yuriy's tool and run phonopy
- Fit

## Teams

- MD: Erik, Han, (Raynol), Prabhath, Jan, Sriram
- QH: Raynol, (Sam), Bharathi, Ahmed, Haitham
- Fit & Yuriy: Sam
- Literature

# Implementation

* https://atomistics.readthedocs.io/en/latest/bulk_modulus_with_gpaw.html#elastic-matrix
* https://github.com/pyiron/atomistics/blob/main/tests/test_elastic_lammpslib_functional.py
* https://github.com/pyiron/pyiron_workflow_atomistics/blob/interstitials/pyiron_workflow_atomistics/dataclass_storage.py
* https://github.com/ligerzero-ai/pyiron_workflow_lammps/blob/main/pyiron_workflow_lammps/engine.py#L21

## Reference

We compare our values with the paper - [M. Krief, et. al., Physical Review E, 103, 063307, 2021](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.103.063307#s4)

Potential used: Copper [Mishin potential](https://www.ctcms.nist.gov/potentials/entry/2001--Mishin-Y-Mehl-M-J-Papaconstantopoulos-D-A-et-al--Cu-1/)

## Imports

In [13]:
from ase.build import bulk
from ase.atoms import Atoms

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from __future__ import annotations

from atomistics.workflows.elastic.workflow import (
    get_tasks_for_elastic_matrix,
    analyse_results_for_elastic_matrix
)

from atomistics.calculators import (
    evaluate_with_lammpslib, 
    get_potential_by_name, 
    calc_molecular_dynamics_npt_with_lammpslib, 
    calc_molecular_dynamics_nvt_with_lammpslib
)

from atomistics.calculators.lammps.libcalculator import (
    calc_static_with_lammpslib, 
    calc_molecular_dynamics_langevin_with_lammpslib
)

In [15]:
from pyiron_base import Project, job

In [16]:
pr = Project("Finite_temp_convergence_4")

## Create bulk sample with a guessed lattice constant

In [17]:
unit_cell = bulk('Cu', 'fcc', a=3.6514, cubic=True) # 4 atoms

In [18]:
repeated_unit_cell = unit_cell.repeat(5) # 500 atoms

In [19]:
potential_name_str = "2001--Mishin-Y--Cu-1--LAMMPS--ipr1"

potential_df = get_potential_by_name(
    potential_name=potential_name_str
)

## 0K Relaxed Structure

In [20]:
def get_relaxed_structure_at_0K(
        structure: Atoms, 
        potential: str, 
        lmp_optimizer_kwargs : dict = None
        ) -> Atoms:
    
    df_pot_selected = get_potential_by_name(
        potential_name=potential
        )
    
    result_dict = evaluate_with_lammpslib(
        task_dict={"optimize_positions_and_volume": structure},
        potential_dataframe=df_pot_selected,
        lmp_optimizer_kwargs=lmp_optimizer_kwargs
        )
    
    structure_relaxed = result_dict['structure_with_optimized_positions_and_volume']

    return structure_relaxed

In [21]:
lmp_optimizer_kwargs={
    'min_style':'cg',
    'ionic_force_tolerance':1e-8,
    'pressure':np.zeros(6) # add anisotropy
}

In [22]:
relaxed_unit_cell = get_relaxed_structure_at_0K(
    unit_cell, # 4 atoms
    potential_name_str, 
    lmp_optimizer_kwargs
)

relaxed_unit_cell # 4 atoms

Atoms(symbols='Cu4', pbc=True, cell=[3.61500008107858, 3.61500008107858, 3.6150000810785805])

## 0K Lattice Constant

In [23]:
def get_lattice_constant_at_0K(
        structure: Atoms, 
        potential: str, 
        lmp_optimizer_kwargs : dict = None
        ) -> float:

    structure_relaxed = get_relaxed_structure_at_0K(
        structure=structure, 
        potential=potential,
        lmp_optimizer_kwargs=lmp_optimizer_kwargs
    )
    
    a_0 = structure_relaxed.get_volume()**(1/3)

    return a_0 # Angstrom

In [24]:
a_0 = get_lattice_constant_at_0K(
    structure=unit_cell, 
    potential=potential_name_str,
    lmp_optimizer_kwargs=lmp_optimizer_kwargs)

a_0 # Angstrom

np.float64(3.61500008107858)

We get the same lattice constant at 0K as the reference paper!

## 0K Elastic Constants

In [25]:
def get_strain_tensor_cubic(
        structure : Atoms, 
        strain : float = 0.005
    ) -> dict:

    deformation_gradient_dict = {
        'C11': np.eye(3,3) + np.array([[ strain,      0,      0], 
                                       [      0,      0,      0],
                                       [      0,      0,      0]]),
        'C12': np.eye(3,3) + np.array([[ strain,      0,      0], 
                                       [      0, strain,      0], 
                                       [      0,      0,      0]]),
        'C44': np.eye(3,3) + np.array([[      0,      0,      0], 
                                       [      0,      0, strain], 
                                       [      0, strain,      0]])
    }

    return deformation_gradient_dict

In [26]:
def get_elastic_constants_from_stress_tensor(
        tensor_dict : dict, 
        strain : float
    ) -> list[float]:

    elastic_constants_list = []

    for constant_str, diff in tensor_dict.items():
        if constant_str == 'C11':
            constant = diff[0, 0] / strain
            elastic_constants_list.append(abs(constant))

        elif constant_str == 'C12':
            sigma33 = diff[2, 2]
            constant = (sigma33/ strain) / 2
            elastic_constants_list.append(abs(constant))

        elif constant_str == 'C44':
            sigma23 = diff[2, 1]
            constant = sigma23 / (2 * strain)
            elastic_constants_list.append(abs(constant))

    return elastic_constants_list

In [27]:
def get_stress_tensor_at_0K(
        structure : Atoms, 
        potential_dataframe : pd.DataFrame, 
        deformation_gradient : np.array
    ):
    
    structure_strained = structure.copy()
    relaxed_cell = np.array(structure_strained.get_cell().tolist())

    strained_cell = deformation_gradient@relaxed_cell
    structure_strained.set_cell(
        strained_cell, 
        scale_atoms=True
    )
    
    relaxed_dict = calc_static_with_lammpslib(
        structure=structure, 
        potential_dataframe=potential_dataframe
    )
    strained_dict = calc_static_with_lammpslib(
        structure=structure_strained,
        potential_dataframe=potential_dataframe
    )

    relaxed_dict['stress_GPa'] = relaxed_dict['stress'] / 10**4
    strained_dict['stress_GPa'] = strained_dict['stress'] / 10**4

    stress_diff = strained_dict['stress_GPa'] - relaxed_dict['stress_GPa']
    
    return stress_diff, relaxed_dict, strained_dict

In [28]:
def calculate_elastic_constants_at_0K(
        structure : Atoms, 
        potential_name : str,
        strain : float = 0.005
    ):

    df_pot_selected = get_potential_by_name(
        potential_name=potential_name
    )

    deformation_gradient_dict = get_strain_tensor_cubic(
        structure=structure, 
        strain=strain
    )
    
    tensor_dict = {}
    for constant_str, deformation_gradient in deformation_gradient_dict.items():
        diff, relaxed_dict, strained_dict = get_stress_tensor_at_0K(
            structure=structure,
            potential_dataframe=df_pot_selected,
            deformation_gradient=deformation_gradient
        )
        
        tensor_dict[constant_str] = diff
        tensor_dict[f'relaxed_dict_{constant_str}'] = relaxed_dict
        tensor_dict[f'strained_dict_{constant_str}'] = strained_dict
    
    elastic_constants_list = get_elastic_constants_from_stress_tensor(
        tensor_dict=tensor_dict, 
        strain=strain
    )

    return elastic_constants_list, tensor_dict

## Reference function to fit elastic constants (Jan + Yury)'s

Requires only `relaxed_unit_cell` and `potential_name_str` from previous cells

In [29]:
def fit_elastic_constants(
        structure: Atoms, 
        potential: str, 
        strains, 
        stresses=None, 
        energies=None):

    task_dict, sym_dict = get_tasks_for_elastic_matrix(
        structure=structure,
        eps_range=0.005,
        num_of_point=5,
        zero_strain_job_name="s_e_0",
        sqrt_eta=True
    )

    potential_df = get_potential_by_name(
        potential_name=potential
    )

    result_dict = evaluate_with_lammpslib(
        task_dict=task_dict,
        potential_dataframe=potential_df,
    )
    
    elastic_dict, sym_dict = analyse_results_for_elastic_matrix(
        output_dict=result_dict,
        sym_dict=sym_dict,
        fit_order=2,
        zero_strain_job_name="s_e_0",
    )

    return elastic_dict

In [30]:
elastic_dict = fit_elastic_constants(
    structure=relaxed_unit_cell,
    potential=potential_name_str,
    strains=None
)

In [31]:
elastic_dict['elastic_matrix']

array([[169.74837327, 123.55258251, 123.55258251,   0.        ,
          0.        ,   0.        ],
       [123.55258251, 169.74837327, 123.55258251,   0.        ,
          0.        ,   0.        ],
       [123.55258251, 123.55258251, 169.74837327,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  76.24914297,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         76.24914297,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  76.24914297]])

In [32]:
elastic_constants_list_reference = [
    elastic_dict['elastic_matrix'][0,0], 
    elastic_dict['elastic_matrix'][0,1], 
    elastic_dict['elastic_matrix'][3,3]
    ]

np.round(elastic_constants_list_reference, 1) # GPa

array([169.7, 123.6,  76.2])

In comparison with the [reference paper](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.103.063307#s4 "M. Krief, et. al., Physical Review E, 103, 063307, 2021"),

$C_{11}$=169.9GPa, $C_{12}$=122.6GPa, and $C_{44}$=76.2GPa

## Finite Temperature equlibiration
* First run NPT to relax volume
* Then equilibriate the cell by running NVT

In [33]:
def equilibriate_structure_at_finite_temperature(
        structure : Atoms, 
        potential : str, 
        temperature : float = 500,
        run : int = 100000,
        thermo : int = 100,
        seed : int = 4928459, 
        cell_scale_value : int = 5,
        thermostat : str = 'langevin'
    ) -> Atoms:

    df_pot_selected = get_potential_by_name(
        potential_name=potential
    )
    
    structure_repeated = structure.repeat(cell_scale_value)

    npt_dict = calc_molecular_dynamics_npt_with_lammpslib(
        structure=structure_repeated,
        potential_dataframe=df_pot_selected,
        Tstart=temperature,
        Tstop=temperature,
        run=run,
        thermo=thermo,
        seed=seed
    )
    
    npt_lattice_constant = (np.mean(npt_dict['volume'][20:]/len(structure_repeated))*len(structure))**(1/3)
    
    # FIXME: Make it for a generic element - something might be wrong here. Need to check error propagation
    # structure_npt = bulk('Cu', a=npt_lattice_constant, cubic=True)
    # structure_repeated_npt = structure_npt.repeat(cell_scale_value)
    
    structure_repeated_npt = structure.copy()
    structure_repeated_npt.set_cell(
        [[npt_lattice_constant,0,0], 
         [0,npt_lattice_constant,0], 
         [0,0,npt_lattice_constant]],
        scale_atoms = True
    )
    structure_repeated_npt = structure_repeated_npt.repeat(cell_scale_value)

    if thermostat == 'nose-hoover':
        nvt_dict = calc_molecular_dynamics_nvt_with_lammpslib(
            structure=structure_repeated_npt,
            potential_dataframe=df_pot_selected,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed
        )
        
    elif thermostat == 'langevin':
        nvt_dict = calc_molecular_dynamics_langevin_with_lammpslib(
            structure=structure_repeated_npt,
            potential_dataframe=df_pot_selected,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed
        )
        
    structure_repeated_nvt = structure_repeated_npt.copy()
    structure_repeated_nvt.set_cell(
        nvt_dict['cell'][-1]
    )
    structure_repeated_nvt.set_positions(
        nvt_dict['positions'][-1]
    )
    structure_repeated_nvt.set_velocities(
        nvt_dict['velocities'][-1]
    )

    return structure_repeated_nvt

## Temperature-dependent Elastic Constants

In [34]:
def get_stress_tensor_at_finite_temperature(
        structure : Atoms, 
        potential_dataframe : pd.DataFrame, 
        deformation_gradient : np.array, 
        temperature : float,
        run : int, 
        thermo : int,
        seed : int,
        thermostat : str
    ):
    
    structure_strained = structure.copy()
    relaxed_cell = np.array(structure_strained.get_cell().tolist())

    strained_cell = deformation_gradient@relaxed_cell
    structure_strained.set_cell(
        strained_cell, 
        scale_atoms=True
    )
    
    if thermostat == 'nose-hoover':
        relaxed_dict = calc_molecular_dynamics_nvt_with_lammpslib(
            structure=structure, 
            potential_dataframe=potential_dataframe,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed,
            disable_initial_velocity=True
        )
        strained_dict = calc_molecular_dynamics_nvt_with_lammpslib(
            structure=structure_strained, 
            potential_dataframe=potential_dataframe,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed,
            disable_initial_velocity=True
        )
    
    elif thermostat == 'langevin':
        relaxed_dict = calc_molecular_dynamics_langevin_with_lammpslib(
            structure=structure, 
            potential_dataframe=potential_dataframe,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed,
            disable_initial_velocity=True
        )
        strained_dict = calc_molecular_dynamics_langevin_with_lammpslib(
            structure=structure_strained, 
            potential_dataframe=potential_dataframe,
            Tstart=temperature,
            Tstop=temperature,
            run=run,
            thermo=thermo,
            seed=seed,
            disable_initial_velocity=True
        )

    relaxed_dict['pressure_GPa'] = relaxed_dict['pressure'] / 10**4
    strained_dict['pressure_GPa'] = strained_dict['pressure'] / 10**4

    stress_diff = -np.mean(strained_dict['pressure_GPa'][20:] - relaxed_dict['pressure_GPa'][20:], axis=0)

    return stress_diff, relaxed_dict, strained_dict

In [ ]:
def calculate_elastic_constants_at_finite_temperature(
        structure : Atoms, # change to unit cell
        cell_scale_value : int,
        potential_name : str, 
        temperature : float = 0, 
        strain : float = 0.005,
        run : int = 10000,
        thermo : int = 100, 
        seed : int = 42, 
        thermostat : str = 'langevin'
    ):

    df_pot_selected = get_potential_by_name(
        potential_name=potential_name
    )

    equilibriated_structure = equilibriate_structure_at_finite_temperature(
        structure=structure,
        potential=potential_name_str, 
        temperature=temperature, 
        seed=seed,
        cell_scale_value=cell_scale_value
    )
    
    deformation_gradient_dict = get_strain_tensor_cubic(
        structure=equilibriated_structure, 
        strain=strain
    )
    
    tensor_dict = {}
    for constant_str, deformation_gradient in deformation_gradient_dict.items():
        diff, relaxed_dict, strained_dict = get_stress_tensor_at_finite_temperature(
            structure=equilibriated_structure, 
            potential_dataframe=df_pot_selected,
            deformation_gradient=deformation_gradient,
            temperature=temperature,
            run=run,
            thermo=thermo,
            seed=seed,
            thermostat=thermostat
        )
        
        tensor_dict[constant_str] = diff
        tensor_dict[f'relaxed_dict_{constant_str}'] = relaxed_dict
        tensor_dict[f'strained_dict_{constant_str}'] = strained_dict
    
    elastic_constants_list = get_elastic_constants_from_stress_tensor(
        tensor_dict=tensor_dict, 
        strain=strain
    )

    return {"elastic_constants": elastic_constants_list, "tensor_dict": tensor_dict}

### Convergence tests

In [38]:
input_params_scale = {
    "cell_scale_value" : [3, 5, 7, 9],
    "run" : [10000],
    "temperature" : [500],
    "strain" : [0.0001],
    "seed": [1357],
    "thermostat" : ["langevin"]
}

In [39]:
from itertools import product
keys = input_params_scale.keys()
values = input_params_scale.values()

for combo in product(*values):
    params = dict(zip(keys, combo))
    print(params)

{'cell_scale_value': 3, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}
{'cell_scale_value': 5, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}
{'cell_scale_value': 7, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}
{'cell_scale_value': 9, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}


In [57]:
input_params_scale = {
    "cell_scale_value" : [2],
    "run" : [100],
    "temperature" : [500],
    "strain" : [0.001],
    "seed": [1357],
    "thermostat" : ["langevin"]
}

In [58]:
def run_convergence(structure, potential_name_str, input_params, project):

    from itertools import product
    from pyiron_base import job
    
    keys = input_params.keys()
    values = input_params.values()

    for combo in product(*values):
        params = dict(zip(keys, combo))
        print(params)

        conv_job = job(calculate_elastic_constants_at_finite_temperature)
        conv_out = conv_job(
            structure = structure,
            potential_name = potential_name_str,
            pyiron_project = project,
            **params
        )

        conv_out.server.queue = "cmmg"
        conv_out.server.cores = 1
        conv_out.server.run_time = 3600

        conv_future = conv_out.pull()

In [59]:
convergence_job = job(run_convergence)

In [60]:
convergence_out = convergence_job(
    structure=relaxed_unit_cell,
    potential_name_str=potential_name_str,
    input_params = input_params_scale,
    project = pr,
    pyiron_project = pr
)

In [61]:
# convergence_out.server.queue = 'cmti'
# convergence_out.server.cores = 40
# convergence_out.server.run_time = 3600

In [62]:
convergence_out

In [63]:
convergence_future = convergence_out.pull()

The job run_convergence_da672396d8fb2c079aa261550d49d85b was saved and received the ID: 30430710
{'cell_scale_value': 2, 'run': 100, 'temperature': 500, 'strain': 0.001, 'seed': 1357, 'thermostat': 'langevin'}
The job calculate_elastic_constants_at_finite_temperature_d308870e78b6dfa9434937fe84b78ca5 was saved and received the ID: 30430711
Queue system id:  18658654


### Results of job decorator trials

* 0- outer - ran local - aborted, with an empty IndexError
* 1- inner - ran local - aborted
* 2- outer - ran local - finished, submitted 3
* 3- inner - submitted to cluster - still running - found `IndexError: tuple index out of range` in `error.out`
* 4- outer - submitted to cluster - aborted
* 5- inner - ran locally on the submitted node - aborted

In [65]:
pr.refresh_job_status()

In [67]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,30402532,aborted,None,run_convergence_8b236c6eee8dbc1ea1bda8643db16c13,/run_convergence_8b236c6eee8dbc1ea1bda8643db16c13,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-10-22 09:47:55.625592,2025-10-22 09:49:54.547541,118.0,pchilaka@cmti001#1,PythonFunctionContainerJob,0.4,None,None
2,30402533,aborted,None,calculate_elastic_constants_at_finite_temperature_3f31ca9f29c781d5aae9a98d2a1f7608,/calculate_elastic_constants_at_finite_temperature_3f31ca9f29c781d5aae9a98d2a1f7608,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-10-22 09:47:55.740714,NaT,NaN,pchilaka@cmti001#1,PythonFunctionContainerJob,0.4,None,None
3,30402546,finished,None,run_convergence_b585bf23f12cf5e996152b9211733b1a,/run_convergence_b585bf23f12cf5e996152b9211733b1a,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-10-22 10:05:50.183369,NaT,NaN,pchilaka@cmti001#1,PythonFunctionContainerJob,0.4,None,None
4,30402547,aborted,None,calculate_elastic_constants_at_finite_temperature_2e4f430ec29d6780fd5be57c1b28b761,/calculate_elastic_constants_at_finite_temperature_2e4f430ec29d6780fd5be57c1b28b761,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-10-22 10:05:50.274458,NaT,NaN,pchilaka@cmti001#40#cmti,PythonFunctionContainerJob,0.4,None,None
5,30402555,aborted,None,run_convergence_85d7818d36a9340e21aeb6de136e7e1e,/run_convergence_85d7818d36a9340e21aeb6de136e7e1e,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-10-22 10:15:41.316619,2025-10-22 10:19:32.703173,231.0,pchilaka@cmti001#40#cmti,PythonFunctionContainerJob,0.4,None,None
1,30402556,aborted,None,calculate_elastic_constants_at_finite_temperature_f5c42c2842e9d3496acfc0553dadc2bf,/calculate_elastic_constants_at_finite_temperature_f5c42c2842e9d3496acfc0553dadc2bf,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-10-22 10:15:47.140769,NaT,NaN,pchilaka@cmti085#1,PythonFunctionContainerJob,0.4,None,None
8,30430700,finished,None,run_convergence_bd9ee943f21bc896fcf8461aa254e78c,/run_convergence_bd9ee943f21bc896fcf8461aa254e78c,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-11-04 15:17:07.672665,NaT,NaN,pchilaka@cmti001#1,PythonFunctionContainerJob,0.4,None,None
6,30430701,finished,None,calculate_elastic_constants_at_finite_temperature_5577a111f30b792e8397249b47a207f5,/calculate_elastic_constants_at_finite_temperature_5577a111f30b792e8397249b47a207f5,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-11-04 15:17:07.816545,NaT,NaN,pchilaka@cmti001#1,PythonFunctionContainerJob,0.4,None,None
7,30430710,finished,None,run_convergence_da672396d8fb2c079aa261550d49d85b,/run_convergence_da672396d8fb2c079aa261550d49d85b,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-11-04 15:25:56.848955,NaT,NaN,pchilaka@cmti001#1,PythonFunctionContainerJob,0.4,None,None
9,30430711,finished,None,calculate_elastic_constants_at_finite_temperature_d308870e78b6dfa9434937fe84b78ca5,/calculate_elastic_constants_at_finite_temperature_d308870e78b6dfa9434937fe84b78ca5,/cmmc/u/,pchilaka/1_Work/1_My_Notebooks/5_Hackathons/hackathon/hackathon/elastic_constants/Finite_temp_convergence_4/,2025-11-04 15:25:56.946120,NaT,NaN,pchilaka@cmti001#1#cmmg,PythonFunctionContainerJob,0.4,None,None


In [68]:
tj = pr.load(30430711)

In [71]:
tj.output

DataContainer({'result': {'elastic_constants': [nan, nan, nan], 'tensor_dict': {'C11': array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]]), 'C12': array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]]), 'C44': array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]]), 'relaxed_dict_C11': {'cell': array([[[7.28739076, 0.        , 0.        ],
        [0.        , 7.28739076, 0.        ],
        [0.        , 0.        , 7.28739076]]]), 'energy_pot': array([-111.16100844]), 'energy_tot': array([-108.8681816]), 'forces': array([[[ 2.76137003, -2.70557783,  1.73358677],
        [ 1.52765299, -2.60668082, -0.8525747 ],
        [ 2.81084238,  3.65007492,  0.5270868 ],
        [ 3.70999681,  0.3647568 , -3.75467657],
        [ 0.02442352,  0.02049265, -3.69702957],
        [ 1.2017556 , -1.42031639,  1.44675337],
        [-0.34704223, -0.83517866,  1.69868602],
        [ 1.07862916, -5.02228061,  3.97523861],
        [ 3.44555729, -3.8686896 ,  3.17801154],
        [-2.72637592,  0.06435013,  4.08083659],
        [-1.24909578, -2.60509258,  3.11053193],
        [-3.44357937,  3.02298941,  3.97258337],
        [ 0.43591667,  1.95912288, -0.72645536],
        [ 0.57840038,  0.62459306, -1.63648256],
        [ 5.2022277 , -0.70516502,  3.11503151],
        [-2.14561383,  2.96683322, -0.27647586],
        [ 1.50017686, -0.72780112,  0.1985675 ],
        [ 2.64759711,  3.48366886,  0.48199821],
        [ 2.9354299 ,  2.65906007,  2.86811395],
        [-1.16938753,  1.13554491,  1.73749545],
        [-0.46780114,  1.35180032, -0.38022799],
        [-2.31638292, -2.67952012,  4.47538993],
        [-0.89403302,  2.59459237, -2.67641567],
        [-0.66794765, -3.44799433,  1.34239719],
        [-3.43623788,  1.78186928,  0.88838251],
        [ 0.32473252, -2.06187014, -1.41489631],
        [ 1.13098874, -2.28257479,  1.36708135],
        [ 0.86495997,  2.66134997, -2.74086784],
        [-3.76064248, -2.29560095,  3.87960065],
        [ 3.09431699,  1.71629882, -1.78002818],
        [-2.96049301, -3.46179533, -2.03499301],
        [ 1.33568473,  1.52044114, -2.49517459]]]), 'positions': array([[[6.74241321, 1.66424521, 0.84416677],
        [6.78561962, 3.42762894, 2.73926469],
        [1.17064126, 1.51259074, 2.70983771],
        [1.20635892, 3.43269933, 0.8102483 ],
        [6.58914286, 1.52576336, 4.43462886],
        [6.39411026, 3.38610844, 6.1884324 ],
        [1.14255405, 1.52742784, 6.30307953],
        [1.13908206, 3.49734884, 4.51707364],
        [6.61888837, 5.10617961, 0.69182893],
        [6.66491197, 6.97397063, 2.55257313],
        [1.18646925, 5.23649279, 2.62243315],
        [1.25211341, 6.98402934, 0.81684348],
        [6.63826119, 5.06931068, 4.6188936 ],
        [6.74880516, 7.16606494, 6.31269631],
        [1.09661674, 5.32127634, 6.26303381],
        [1.1309566 , 6.95603748, 4.45832646],
        [3.06177772, 1.52473742, 0.90920533],
        [2.85907504, 3.31932923, 2.54570637],
        [4.93601755, 1.34325244, 2.6508105 ],
        [4.8210906 , 3.32633106, 0.89029179],
        [2.94326888, 1.5589346 , 4.48694152],
        [2.93727535, 3.30954444, 6.24199429],
        [4.77812427, 1.53690077, 6.50474829],
        [4.58996705, 3.20524224, 4.31987239],
        [3.20151281, 5.17704032, 1.00613049],
        [2.99402285, 7.08135381, 2.6532249 ],
        [5.12444444, 5.15849704, 2.92288628],
        [5.03671964, 7.0366851 , 0.86601541],
        [2.93583279, 5.14358021, 4.34293803],
        [3.06293146, 7.01856922, 6.4440596 ],
        [4.72163844, 5.12347021, 6.31868972],
        [4.8287532 , 6.9541656 , 4.61152713]]]), 'pressure': array([[[ 1191.02893716,  2760.50635754,  4806.45067774],
        [ 2760.50635754,  3343.05323113, -1802.92679018],
        [ 4806.45067774, -1802.92679018, -3344.30473448]]]), 'pressure_GPa': array([[[ 0.11910289,  0.27605064,  0.48064507],
        [ 0.27605064,  0.33430532, -0.18029268],
        [ 0.48064507, -0.18029268, -0.33443047]]]), 'tempera

In [69]:
tj.output.result['elastic_constants']

[nan, nan, nan]

In [48]:
tj.content['storage']

AttributeError: 'Atoms' object has no attribute 'list_nodes'

AttributeError: 'Atoms' object has no attribute 'list_nodes'

DataContainer({'input': HDFStub({'groups': ['input__index_0', 'output__index_1'], 'nodes': ['DICT_VERSION', 'HDF_VERSION', 'NAME', 'OBJECT', 'READ_ONLY', 'TYPE', 'VERSION']}, input__index_0), 'output': HDFStub({'groups': ['input__index_0', 'output__index_1'], 'nodes': ['DICT_VERSION', 'HDF_VERSION', 'NAME', 'OBJECT', 'READ_ONLY', 'TYPE', 'VERSION']}, output__index_1)})

### Running directly

In [ ]:
# def run_convergence(structure, potential_name_str, input_params):

#     from itertools import product
#     from pyiron_base import job
    
#     keys = input_params.keys()
#     values = input_params.values()

#     for combo in product(*values):
#         params = dict(zip(keys, combo))
#         print(params)

#         elastic_list, tensor_dict = calculate_elastic_constants_at_finite_temperature(
#             structure = structure,
#             potential_name = potential_name_str,
#             **params
#         )

#     return elastic_list, tensor_dict

In [32]:
# elastic_list, tensor_dict = run_convergence(relaxed_unit_cell, potential_name_str, input_params_scale)

{'cell_scale_value': 4, 'run': 10000, 'temperature': 500, 'strain': 0.0001, 'seed': 1357, 'thermostat': 'langevin'}

LAMMPS INPUT TEMPLATE THAT IS BEING RUN: 
 thermo_style custom step temp pe etotal pxx pxy pxz pyy pyz pzz vol
thermo_modify format float %20.15g
timestep 0.001
thermo 100
velocity all create $(2 * 500) 1357 dist gaussian
fix integration all nve
fix ensemble all langevin 500 500 0.1 1357

LAMMPS INPUT TEMPLATE THAT IS BEING RUN: 
 thermo_style custom step temp pe etotal pxx pxy pxz pyy pyz pzz vol
thermo_modify format float %20.15g
timestep 0.001
thermo 100
fix integration all nve
fix ensemble all langevin 500 500 0.1 1357

LAMMPS INPUT TEMPLATE THAT IS BEING RUN: 
 thermo_style custom step temp pe etotal pxx pxy pxz pyy pyz pzz vol
thermo_modify format float %20.15g
timestep 0.001
thermo 100
fix integration all nve
fix ensemble all langevin 500 500 0.1 1357

LAMMPS INPUT TEMPLATE THAT IS BEING RUN: 
 thermo_style custom step temp pe etotal pxx pxy pxz pyy pyz pzz vol
th

## Pyiron table

In [ ]:
def db_filter_function(job_table):
    return (job_table.status == "finished") & (job_table.hamilton == "PythonFunctionContainerJob")

def job_filter_function(job):
    return (job.status == "finished") & ("calculate" in job.name)

def get_input_structure(job):
    return job.project_hdf5['input']['data']['structure']

In [ ]:
table = pr.create.table("Convergence_pyiron_table", delete_existing_job=True)

# table.db_filter_function = db_filter_function
table.filter_function = job_filter_function

table.add["input_structure"] = get_input_structure

table.run(delete_existing_job=True)

In [ ]:
table.get_dataframe()